In [1]:
# Data
import numpy as np

with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [2]:
# Model
import impl.layer as l
from impl.loss import *

class LSTM:
    
    def __init__(self, D, H, L, char2idx, idx2char):
        self.D = D
        self.H = H
        self.L = L
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.vocab_size = len(char2idx)
        self.losses = {'train':[], 'smooth train':[]}
        
        # Model parameters wights and biases
        Z = H + D
        m = dict(
            Wf=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wi=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wc=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wo=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wy=np.random.randn(H, D) / np.sqrt(H / 2.),
            bf=np.zeros((1, H)),
            bi=np.zeros((1, H)),
            bc=np.zeros((1, H)),
            bo=np.zeros((1, H)),
            by=np.zeros((1, D)))

        self.model = []
        for _ in range(self.L):
            self.model.append(m)

    def initial_state(self):
        return (np.zeros((1, self.H)), np.zeros((1, self.H)))

    def forward(self, X, h, m):
        Wf, Wi, Wc, Wo, Wy = m['Wf'], m['Wi'], m['Wc'], m['Wo'], m['Wy']
        bf, bi, bc, bo, by = m['bf'], m['bi'], m['bc'], m['bo'], m['by']

        h_in, c_in = h
        X_in = X.copy()

        X = np.column_stack((h_in, X_in))

        hf, hf_cache = l.fc_forward(X, Wf, bf)
        hf, hf_sigm_cache = l.sigmoid_forward(hf)

        hi, hi_cache = l.fc_forward(X, Wi, bi)
        hi, hi_sigm_cache = l.sigmoid_forward(hi)

        hc, hc_cache = l.fc_forward(X, Wc, bc)
        hc, hc_tanh_cache = l.tanh_forward(hc)

        ho, ho_cache = l.fc_forward(X, Wo, bo)
        ho, ho_sigm_cache = l.sigmoid_forward(ho)

        c = (hf * c_in) + (hi * hc)
        c, c_tanh_cache = l.tanh_forward(c)

        h = ho * c
        
        h_ = (h, c)
        y, y_cache = l.fc_forward(h, Wy, by)
        
        cache = (c_in, hf, hf_cache, hf_sigm_cache, hi, hi_cache, hi_sigm_cache, hc, hc_cache, hc_tanh_cache, ho, ho_cache, ho_sigm_cache, c, c_tanh_cache, y_cache)
        
        return y, h_, cache

    def backward(self, dy, dh, cache):
        c_in, hf, hf_cache, hf_sigm_cache, hi, hi_cache, hi_sigm_cache, hc, hc_cache, hc_tanh_cache, ho, ho_cache, ho_sigm_cache, c, c_tanh_cache, y_cache = cache

        dh_out, dc_out = dh

        dh, dWy, dby = l.fc_backward(dy, y_cache)
        dh += dh_out

        dc = ho * dh
        dc = l.tanh_backward(dc, c_tanh_cache)
        dc += dc_out
        dc_in = hf * dc

        dho = c * dh
        dho = l.sigmoid_backward(dho, ho_sigm_cache)
        dXo, dWo, dbo = l.fc_backward(dho, ho_cache)

        dhc = hi * dc
        dhc = l.tanh_backward(dhc, hc_tanh_cache)
        dXc, dWc, dbc = l.fc_backward(dhc, hc_cache)

        dhi = hc * dc
        dhi = l.sigmoid_backward(dhi, hi_sigm_cache)
        dXi, dWi, dbi = l.fc_backward(dhi, hi_cache)

        dhf = c_in * dc
        dhf = l.sigmoid_backward(dhf, hf_sigm_cache)
        dXf, dWf, dbf = l.fc_backward(dhf, hf_cache)

        dX = dXo + dXc + dXi + dXf
        dh_in = dX[:, :self.H]
        dX_in = dX[:, self.H:]
        
        dh = (dh_in, dc_in)
        dX = dX_in

        grad = dict(Wf=dWf, Wi=dWi, Wc=dWc, Wo=dWo, Wy=dWy, bf=dbf, bi=dbi, bc=dbc, bo=dbo, by=dby)

        return dX, dh, grad
            
    def train_forward(self, X_train, h_):
        ys, caches = [], []
        #         h_init = h.copy()
        h, c = h_
        h_init = (h.copy(), c.copy())
        h = []
        for _ in range(self.L):
            h.append(h_init)
            caches.append([])
            
        for X in X_train:
            X_one_hot = np.zeros(self.D)
            X_one_hot[X] = 1.
            X = X_one_hot.reshape(1, -1)
            for layer in range(self.L):
                y, h[layer], cache = self.forward(X, h[layer], self.model[layer])
                caches[layer].append(cache)
                X = y.copy()
            ys.append(y)
            
        return ys, caches
    
    def loss_function(self, y_train, ys):
        loss, dys = 0.0, []

        for y_pred, y in zip(ys, y_train):
            loss += cross_entropy(y_pred, y)
            dy = dcross_entropy(y_pred, y)
            dys.append(dy)
            
        return loss, dys

    def train_backward(self, dys, caches):
        dh, grad, grads = [], [], []
        for layer in range(self.L):
            dh.append((np.zeros((1, self.H)), np.zeros((1, self.H))))
            grad.append({key: np.zeros_like(val) for key, val in self.model[layer].items()})
            grads.append({key: np.zeros_like(val) for key, val in self.model[layer].items()})
            
        dXs = []
        for t in reversed(range(len(dys))):
            dy = dys[t]
            for layer in reversed(range(self.L)):
                dX, dh[layer], grad[layer] = self.backward(dy, dh[layer], caches[layer][t])
                for key in grad[layer].keys():
                    grads[layer][key] += grad[layer][key]
                dy = dX.copy()
            dXs.append(dX)
                
        return dXs, grads
    
    def test(self, X_seed, h_, size):
        chars = [self.idx2char[X_seed]]
        idx_list = list(range(self.vocab_size))
        X = X_seed
        
        #         h_init = h.copy()
        h, c = h_
        h_init = (h.copy(), c.copy())
        h = []
        for _ in range(self.L):
            h.append(h_init)

        for _ in range(size):
            X_one_hot = np.zeros(self.D)
            X_one_hot[X] = 1.
            X = X_one_hot.reshape(1, -1)
            for layer in range(self.L):
                y, h[layer], _ = self.forward(X, h[layer], self.model[layer])
                X = y.copy()
                
            prob = l.softmax(y)
            idx = np.random.choice(idx_list, p=prob.ravel())
            chars.append(self.idx2char[idx])
            X = idx

        return ''.join(chars)

In [3]:
def get_minibatch(X, y, minibatch_size, shuffle):
    minibatches = []

    # for i in range(0, X.shape[0], minibatch_size):
    for i in range(0, X.shape[0] - minibatch_size + 1, 1):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]
        minibatches.append((X_mini, y_mini))

    return minibatches

def adam_rnn(nn, X_train, y_train, alpha, mb_size, n_iter, print_after):
    M, R = [], []
    for layer in range(nn.L):
        M.append({key: np.zeros_like(val) for key, val in nn.model[layer].items()})
        R.append({key: np.zeros_like(val) for key, val in nn.model[layer].items()})
        
    beta1 = .99
    beta2 = .999
    state = nn.initial_state()
    smooth_loss = 1
    minibatches = get_minibatch(X_train, y_train, mb_size, shuffle=False)

    # Epochs
    for iter in range(1, n_iter + 1):
        # Minibatches
        for idx in range(len(minibatches)):
            X_mini, y_mini = minibatches[idx]
            ys, caches = nn.train_forward(X_mini, state)
            loss, dys = nn.loss_function(y_mini, ys)
            _, grads = nn.train_backward(dys, caches)
            nn.losses['train'].append(loss)
            smooth_loss = (0.999 * smooth_loss) + (0.001 * loss)
            nn.losses['smooth train'].append(smooth_loss)
            
            # Updating the model parameters
            for layer in range(nn.L):
                for key in grads[layer].keys(): #key, value: items, dict={}
                    M[layer][key] = l.exp_running_avg(M[layer][key], grads[layer][key], beta1)
                    R[layer][key] = l.exp_running_avg(R[layer][key], grads[layer][key]**2, beta2)

                    m_k_hat = M[layer][key] / (1. - (beta1**(iter)))
                    r_k_hat = R[layer][key] / (1. - (beta2**(iter)))

                    nn.model[layer][key] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + l.eps)
                
        # Print loss and test sample
        if iter % print_after == 0:
            print('Iter-{} loss: {:.4f}'.format(iter, loss))
            sample = nn.test(X_mini[0], state, size=200) # time_step=mb_size
            print(sample)
    
    return nn

In [ ]:
# Hyper-parameters
time_step = 100 # width, minibatch size and test sample size as well
num_layers = 2 # depth
n_iter = 300 # epochs
alpha = 1e-4 # learning_rate
p_dropout = 0.95 # q=1-p, q=keep_prob and p=dropout.
print_after = 1 # n_iter//10 # print training loss, valid, and test
num_hidden_units = 64 # num_hidden_units in hidden layer
num_input_units = len(char_to_idx) # vocab_size = len(char_to_idx)

# Build the network and learning it or optimizing it using SGD
net = LSTM(D=num_input_units, H=num_hidden_units, L=num_layers, char2idx=char_to_idx, idx2char=idx_to_char)

# Start learning using BP-SGD-ADAM
adam_rnn(nn=net, X_train=X, y_train=y, alpha=alpha, mb_size=time_step, n_iter=n_iter, print_after=print_after)

# # Display the learning curve and losses for training, validation, and testing
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

plt.plot(net.losses['train'], label='Train loss')
plt.plot(net.losses['smooth train'], label='Train smooth loss')
plt.legend()
plt.show()

Iter-1 loss: 331.1303
 sshaisynth  unm  etktorkd  enrntlor0shriina rgpey meyy seineasndacrro  n uivi aen y  ietaf cahunyans dnnkG  uyrhc her ninxetlr atant ashsehfaIaNe ixb e ni8t arra es ,aein wh incfetIiinnexa s so5hgi At
Iter-2 loss: 319.6571
 dat ses, WnhIaasld 2iRsa tf nst hcskn fnbiiet te ta h tdifhibeddh a k i5 irne xexruteulto iwvirod an xiipys l crhi ti ny net Dxplt xbart t ileenpIet e om s sh hht ahoaoet, ss tih IhfoC, s bd1ctttelonj
Iter-3 loss: 302.9954
 ieshmrxluit -nees 5if xnrlh ts nyeJuliys jde yoabrh k iiv ti. ilesr iwy nEtned fs rntt urcngiy tis p-txutiy in Inouvoky tte rri. seyh -ptents ianeh eesedd peaimi' th sNctyy y sehyt -tte br Gtoie ns at
Iter-4 loss: 304.6708
 , letlef G an ath 日nndh if inge2–sh f  ab poHan Jitn W  f R1nn xin  1aGnhe Ht;3e s riblyd f  iIvif    alh-o JfAs mpc7naf Df ip Wab, e–af f  aIe  paiAt v3  arrs In i7c 0-mjf owbb 9pnvy 1Gpnf 本aWWl- sor
Iter-5 loss: 242.7744
 pextat Cothe lhensi, und lanhy Re bad tate Cinns tey leth uns terxrng retutiFt in

Iter-38 loss: 148.7369
 Wigieng fy moun to hond first lisicarch runseladest in the north lowesked, largest ereollowes that largest Ireopectire in istin-revighe for. ERfuthh milltn of Cctority holankhhily Wourd Hinar power la
Iter-39 loss: 140.1436
 in the Gloub Peades, in the Grofset in ragedst unolivand a Deuthe is rofelect op is thming lobsed liwal the War aed cone the sixth largest mill mation is in the thirst country in East ghontrkited in t
Iter-40 loss: 141.9873
 Ni9prng, is the highest-ranked bony forr w rof-ipenest-reften a whor, whith esed-ralgentured unti. Lowache, byk wor world. Japan has offecirilegeside thindst efterce bilth largest militar and conse in
Iter-41 loss: 132.8979
 in the north liet exping powet-laveternd in the segtest reates covelation the nourthe bon the third lirterth country wirth lichatime a pertokea, stwoch rinit poweturceg, of icronfologlitar of iseledes
Iter-42 loss: 132.2083
 the world, and fegtentic-tentitutrend country in the Paman is amperorcens Eo